# Setup

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import os
from pathlib import Path
import statistics

import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, RepeatedStratifiedKFold

import catboost as cb
import optuna

In [3]:
AUTO_FIND_HYPERPARAMETERS = True
MANUAL_FIND_HYPERPARAMETERS = False
EVALUATE_ON_VAL = False
USE_CATBOOST_CLASSIFIER = False
USE_TASK_FEATURES = False
USE_LENGTH_FEATURES = True
USE_NEUTRAL_PROBA_FEATURE = False
RUN_NAME = 'test_article'

INPUT_DATA_PATH = './data/'
REFERENCE_PATH = './data/test/'
OUTPUT_DATA_PATH = './output/'
SUBMISSION_PATH = f'./output/catboost/{RUN_NAME}/'
RESULT_PATH = f'./output/catboost/{RUN_NAME}/test.score.txt'
MODELS = [
    'FacebookAI-roberta-large-mnli',
    # 'FacebookAI-roberta-large-mnli_inverse',
    'deberta-selfchecknli',
    # 'deberta-selfchecknli_inverse',
    'google_t5_xxl_true_nli_mixture',
    # 'google_t5_xxl_true_nli_mixture_inverse',
    'sentence-transformers-nli-roberta-large',
    # 'openchat',
    'microsoft-deberta-base-mnli', 
    'microsoft-deberta-large-mnli', 
    'microsoft-deberta-xlarge-mnli',
    'microsoft-deberta-v2-xlarge-mnli',
    'microsoft-deberta-v2-xxlarge-mnli',
    ]

RANDOM_SEED = 424242
TASKS = ['agnostic', 'aware']

Path(SUBMISSION_PATH).mkdir(parents=True, exist_ok=True)

# Load data

In [4]:
X = {}
y = {}
for sample in ['val', 'test']:
    for task in TASKS:
        X[f'{sample}-{task}'] = []
        
        # Load 'task' and 'length' features
        file = f'{INPUT_DATA_PATH}{sample}/{sample}.model-{task}.json'
        df = pd.read_json(file, orient='records')
        if sample == 'test':
            df.set_index('id', drop=True, inplace=True)
        if USE_TASK_FEATURES:
            if MANUAL_FIND_HYPERPARAMETERS:
                X[f'{sample}-{task}'].append(df['task'])
            elif AUTO_FIND_HYPERPARAMETERS:
                df['DM'] = df['task'].map({'DM': 1, 'MT': 0, 'PG': 0})
                df['MT'] = df['task'].map({'DM': 0, 'MT': 1, 'PG': 0})
                X[f'{sample}-{task}'].append(df[['DM', 'MT']])
        if USE_LENGTH_FEATURES:
            df['hyp_len'] = df['hyp'].str.len()
            df['tgt_len'] = df['tgt'].str.len()
            df['src_len'] = df['src'].str.len()
            X[f'{sample}-{task}'].append(df[['hyp_len', 'tgt_len', 'src_len']])
        print(file)
        
        # Load y
        df.rename(columns={"p(Hallucination)": "p"}, inplace=True)
        y[f'{sample}-{task}'] = df[['label', 'p']]

        for model in MODELS:
            # Load features from transformer-based models
            file = f'{OUTPUT_DATA_PATH}{model}/{sample}/{sample}.model-{task}.csv'
            df = pd.read_csv(file)
            if sample == 'test':
                df.set_index('id', drop=True, inplace=True)
            if USE_NEUTRAL_PROBA_FEATURE:
                if "p(Entl)" in df:
                    df['p(Neutr)'] = 1 - df['p(Contr)'] - df['p(Entl)']
                else:
                    df['p(Entl)'] = 1 - df['p(Contr)']
            df.rename(columns={
                "p(Contr)": "p(Contr)_" + model,
                "p(Entl)": "p(Entl)_" + model,
                "p(Neutr)": "p(Neutr)_" + model,
                }, inplace=True)
            X[f'{sample}-{task}'].append(df)
            print(file)
        
        # Concatenate X dataframes
        X[f'{sample}-{task}'] = pd.concat(X[f'{sample}-{task}'], axis=1)

./data/val/val.model-agnostic.json
./output/FacebookAI-roberta-large-mnli/val/val.model-agnostic.csv
./output/deberta-selfchecknli/val/val.model-agnostic.csv
./output/google_t5_xxl_true_nli_mixture/val/val.model-agnostic.csv
./output/sentence-transformers-nli-roberta-large/val/val.model-agnostic.csv
./output/microsoft-deberta-base-mnli/val/val.model-agnostic.csv
./output/microsoft-deberta-large-mnli/val/val.model-agnostic.csv
./output/microsoft-deberta-xlarge-mnli/val/val.model-agnostic.csv
./output/microsoft-deberta-v2-xlarge-mnli/val/val.model-agnostic.csv
./output/microsoft-deberta-v2-xxlarge-mnli/val/val.model-agnostic.csv
./data/val/val.model-aware.json
./output/FacebookAI-roberta-large-mnli/val/val.model-aware.csv
./output/deberta-selfchecknli/val/val.model-aware.csv
./output/google_t5_xxl_true_nli_mixture/val/val.model-aware.csv
./output/sentence-transformers-nli-roberta-large/val/val.model-aware.csv
./output/microsoft-deberta-base-mnli/val/val.model-aware.csv
./output/microsoft

In [5]:
# Print dataframe names
print('For X data:')
for df_name in X:
    print('\t', df_name)
print()
print('For y data:')
for df_name in y:
    print('\t', df_name)

For X data:
	 val-agnostic
	 val-aware
	 test-agnostic
	 test-aware

For y data:
	 val-agnostic
	 val-aware
	 test-agnostic
	 test-aware


In [6]:
X['val-agnostic']

,hyp_len,tgt_len,src_len,p(Contr)_FacebookAI-roberta-large-mnli,p(Contr)_deberta-selfchecknli,p(Contr)_google_t5_xxl_true_nli_mixture,p(Contr)_sentence-transformers-nli-roberta-large,p(Contr)_microsoft-deberta-base-mnli,p(Entl)_microsoft-deberta-base-mnli,p(Contr)_microsoft-deberta-large-mnli,p(Entl)_microsoft-deberta-large-mnli,p(Contr)_microsoft-deberta-xlarge-mnli,p(Entl)_microsoft-deberta-xlarge-mnli,p(Contr)_microsoft-deberta-v2-xlarge-mnli,p(Entl)_microsoft-deberta-v2-xlarge-mnli,p(Contr)_microsoft-deberta-v2-xxlarge-mnli,p(Entl)_microsoft-deberta-v2-xxlarge-mnli
0,41,36,316,0.001844,0.002097,0,0.053903,0.001249,0.918769,0.000930,0.925807,0.001269,0.937311,0.001208,0.956369,0.002282,0.913186
1,32,1,451,0.289209,0.643224,1,0.955537,0.043280,0.451597,0.053752,0.202548,0.056093,0.335795,0.026490,0.200199,0.034473,0.020775
2,34,36,181,0.993463,0.707190,1,0.107394,0.367452,0.423152,0.843042,0.030436,0.409416,0.335741,0.669810,0.130411,0.604200,0.209205
3,17,100,149,0.094564,0.008859,0,0.428500,0.045262,0.933873,0.010749,0.969606,0.004535,0.980909,0.052765,0.911118,0.007620,0.975416
4,39,63,441,0.013538,0.004337,0,0.059323,0.209675,0.284467,0.030628,0.928012,0.008921,0.891682,0.003898,0.962758,0.013182,0.932905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,32,39,30,0.001673,0.000138,0,0.014917,0.000238,0.997373,0.000179,0.998278,0.000507,0.994063,0.000151,0.998987,0.000825,0.995205
495,42,46,48,0.004702,0.000263,0,0.024462,0.002412,0.963516,0.022022,0.900231,0.024779,0.931660,0.005710,0.983528,0.001241,0.991755
496,43,42,43,0.001838,0.000291,0,0.016321,0.000273,0.996779,0.000256,0.996457,0.000437,0.992136,0.000263,0.998185,0.000795,0.994654
497,28,31,22,0.000960,0.000053,0,0.042848,0.000354,0.995894,0.000149,0.997803,0.000396,0.993571,0.000123,0.998568,0.000770,0.994802


In [7]:
y['val-agnostic']

,label,p
0,Not Hallucination,0.2
1,Hallucination,0.8
2,Not Hallucination,0.0
3,Not Hallucination,0.2
4,Not Hallucination,0.0
...,...,...
494,Not Hallucination,0.0
495,Not Hallucination,0.2
496,Not Hallucination,0.0
497,Not Hallucination,0.0


# CatBoost Classifier
## Find parameters

In [8]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 400),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.1),
        'depth': trial.suggest_int('depth', 6, 12),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-2, 10),
        'border_count': trial.suggest_int('border_count', 5, 200),
        'verbose': False,
        'random_seed': RANDOM_SEED
    }

    model = cb.CatBoostClassifier(**params)

    # Cross validation
    sample = 'val'
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)
    accuracy = cross_val_score(model, X[f'{sample}-{task}'], y[f'{sample}-{task}']['label'], cv=cv, scoring='accuracy').mean()

    return accuracy


best_params_dict = {}
if USE_CATBOOST_CLASSIFIER and AUTO_FIND_HYPERPARAMETERS:
    for task in TASKS:
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=100)
        best_params_dict[task] = study.best_params

elif USE_CATBOOST_CLASSIFIER:
    best_params_dict['agnostic'] = {'iterations': 216, 'learning_rate': 0.0102019005543733, 'depth': 12, 'l2_leaf_reg': 0.09976943036124933, 'border_count': 39}
    best_params_dict['aware'] = {'iterations': 129, 'learning_rate': 0.005371920324746337, 'depth': 9, 'l2_leaf_reg': 0.6911690977933583, 'border_count': 186}

In [9]:
if USE_CATBOOST_CLASSIFIER:
    for task in TASKS:
        print("Best Params:", task.ljust(15), best_params_dict[task])

## Evaluate on validation sample

In [10]:
if USE_CATBOOST_CLASSIFIER and EVALUATE_ON_VAL:
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=RANDOM_SEED)
    sample = 'val'
    mean_accuracy_dict = {}
    for task in TASKS:
        model = cb.CatBoostClassifier(**best_params_dict[task])
        accuracy = []
        for i, (train_index, test_index) in enumerate(cv.split(X[f'{sample}-{task}'], y[f'{sample}-{task}']['label'])):
            # Train
            X_train = X[f'{sample}-{task}'].loc[train_index]
            y_train = y[f'{sample}-{task}']['label'].loc[train_index]
            model.fit(X_train, y_train)

            # Predict
            X_test = X[f'{sample}-{task}'].loc[test_index]
            y_test = y[f'{sample}-{task}']['label'].loc[test_index]
            y_pred = model.predict(X_test)

            # Calc metric
            accuracy.append(accuracy_score(y_test, y_pred))
            
        # Calc average metric
        mean_accuracy = statistics.fmean(accuracy)
        mean_accuracy_dict[task] = mean_accuracy

In [11]:
if USE_CATBOOST_CLASSIFIER and EVALUATE_ON_VAL:
    for task in TASKS:
        print('Accuracy:', task.ljust(15), mean_accuracy_dict[task])

## Train on 100 % of data

In [12]:
if USE_CATBOOST_CLASSIFIER:
    sample = 'val'
    models = {}
    for task in TASKS:
        model = cb.CatBoostClassifier(**best_params_dict[task])
        model.fit(X[f'{sample}-{task}'], y[f'{sample}-{task}']['label'])
        models[task] = model

## Predict

In [13]:
if USE_CATBOOST_CLASSIFIER:
    sample = 'test'
    preds_label = {}
    for task in TASKS:
        preds_label[task] = models[task].predict(X[f'{sample}-{task}'])
        print(f'{sample}-{task} - done!')

# CatBoost Regressor
## Manual search hyperparameters

In [14]:
if MANUAL_FIND_HYPERPARAMETERS:
    task = 'agnostic'

    # initialize Pool
    train_pool = cb.Pool(
        X[f'val-{task}'], 
        y[f'val-{task}']['p'], 
        cat_features=['task'],
        )
    test_pool = cb.Pool(
        X[f'test-{task}'], 
        cat_features=['task'],
        )

    params = {
        'learning_rate': 0.02,
        'iterations': 1000,
        'depth': 4,
        'early_stopping_rounds': 350,
        'verbose': False,
        'loss_function': 'RMSE',
        'task_type': 'GPU',
    }

    scores = cb.cv(
        train_pool,
        params,
        fold_count=10,
        plot=True,
    )

    display(scores)
    scores['test-RMSE-mean'].plot(ylim=[0.21, 0.24], grid=True);

In [15]:
if MANUAL_FIND_HYPERPARAMETERS:
    iter = 480
    print(scores['test-RMSE-mean'][iter])
    print(scores['test-RMSE-std'][iter])

In [16]:
if MANUAL_FIND_HYPERPARAMETERS:
    pd.DataFrame({
        'lr': [0.07, 0.07, 0.05, 0.05, 0.04, 0.03, 0.03, 0.02, 0.02],
        'iter': [140, 200, 190, 190, 190, 440, 240, 480, 600],
        'depth': [4, 6, 4, 6, 6, 4, 6, 4, 6],
        'RMSE-mean': [0.221920, 0.222109, 0.222369, 0.221364, 0.221469, 0.221713, 0.223381, 0.221618, 0.223620],
        'RMSE-std':  [0.024485, 0.026171, 0.023741, 0.025904, 0.023871, 0.025716, 0.024253, 0.025951, 0.024863],
    })

In [17]:
if MANUAL_FIND_HYPERPARAMETERS:
    model = cb.CatBoostRegressor(**params)
    model.fit(
        train_pool,
        # eval_set=,
        plot=True,
    )

    preds = model.predict(test_pool)
    print(preds)

## Auto search hyperparameters

In [18]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 400),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'depth': trial.suggest_int('depth', 4, 14),
        # 'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        # 'border_count': trial.suggest_int('border_count', 5, 200),
        'verbose': False,
        'random_seed': RANDOM_SEED
    }

    model = cb.CatBoostRegressor(**params)

    # Cross validation
    sample = 'val'
    cv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)
    mse = - cross_val_score(model, X[f'{sample}-{task}'], y[f'{sample}-{task}']['p'], cv=cv, scoring='neg_mean_squared_error').mean()

    return mse


best_params_dict = {}
if AUTO_FIND_HYPERPARAMETERS:
    for task in TASKS:
        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=100)
        best_params_dict[task] = study.best_params
        
else:
    best_params_dict['agnostic'] = {'iterations': 394, 'learning_rate': 0.021608465544161754, 'depth': 4, 'l2_leaf_reg': 0.007156537267301533, 'border_count': 98}
    best_params_dict['aware'] = {'iterations': 135, 'learning_rate': 0.025526913154005487, 'depth': 4, 'l2_leaf_reg': 0.12536020060626074, 'border_count': 118}

[I 2024-04-02 07:49:42,262] A new study created in memory with name: no-name-d0043247-272d-481c-ab60-6e287f4580c7


/tmp/ipykernel_637554/1172307581.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),


[I 2024-04-02 07:50:00,770] Trial 0 finished with value: 0.05264825743571246 and parameters: {'iterations': 386, 'learning_rate': 0.01667442575532042, 'depth': 9}. Best is trial 0 with value: 0.05264825743571246.


/tmp/ipykernel_637554/1172307581.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),


[I 2024-04-02 07:50:11,731] Trial 1 finished with value: 0.052488499124595744 and parameters: {'iterations': 311, 'learning_rate': 0.016836294833420033, 'depth': 8}. Best is trial 1 with value: 0.052488499124595744.


...

[I 2024-04-02 08:39:46,313] Trial 97 finished with value: 0.05184508831672511 and parameters: {'iterations': 376, 'learning_rate': 0.009275794073991211, 'depth': 7}. Best is trial 64 with value: 0.05151694436417945.


/tmp/ipykernel_637554/1172307581.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),


[I 2024-04-02 08:39:50,195] Trial 98 finished with value: 0.051911808100259714 and parameters: {'iterations': 398, 'learning_rate': 0.01242400316154969, 'depth': 5}. Best is trial 64 with value: 0.05151694436417945.


/tmp/ipykernel_637554/1172307581.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),


[I 2024-04-02 08:39:54,509] Trial 99 finished with value: 0.05194253524059174 and parameters: {'iterations': 324, 'learning_rate': 0.014331812453527703, 'depth': 6}. Best is trial 64 with value: 0.05151694436417945.


In [19]:
for task in TASKS:
    print("Best Params:", task.ljust(15), best_params_dict[task])

Best Params: agnostic        {'iterations': 356, 'learning_rate': 0.02855915846133446, 'depth': 5}
Best Params: aware           {'iterations': 317, 'learning_rate': 0.012105132480904755, 'depth': 9}


## Evaluate on validation sample

In [20]:
if EVALUATE_ON_VAL:
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=RANDOM_SEED)
    sample = 'val'
    mean_corr_dict = {}
    for task in TASKS:
        model = cb.CatBoostRegressor(**best_params_dict[task])
        correlation = []
        for i, (train_index, test_index) in enumerate(cv.split(X[f'{sample}-{task}'], y[f'{sample}-{task}']['label'])):
            # Train
            X_train = X[f'{sample}-{task}'].loc[train_index]
            y_train = y[f'{sample}-{task}']['p'].loc[train_index]
            model.fit(X_train, y_train)

            # Predict
            X_test = X[f'{sample}-{task}'].loc[test_index]
            y_test = y[f'{sample}-{task}']['p'].loc[test_index]
            y_pred = model.predict(X_test)

            # Postprocessing
            np.clip(y_pred, 0.0, 1.0, out=y_pred)

            # Calc metric
            correlation.append(spearmanr(y_pred, y_test)[0])
            
        # Calc average metric
        mean_corr = statistics.fmean(correlation)
        mean_corr_dict[task] = mean_corr

In [21]:
if EVALUATE_ON_VAL:
    for task in TASKS:
        print('Correlation:', task.ljust(15), mean_corr_dict[task])

## Train on 100 % of data

In [22]:
sample = 'val'
models = {}
for task in TASKS:
    model = cb.CatBoostRegressor(**best_params_dict[task])
    model.fit(X[f'{sample}-{task}'], y[f'{sample}-{task}']['p'])
    models[task] = model

0:	learn: 0.3465954	total: 977us	remaining: 347ms
1:	learn: 0.3416215	total: 1.86ms	remaining: 329ms
2:	learn: 0.3370168	total: 2.71ms	remaining: 319ms
3:	learn: 0.3324188	total: 3.48ms	remaining: 306ms
4:	learn: 0.3280985	total: 4.24ms	remaining: 298ms
5:	learn: 0.3238491	total: 4.97ms	remaining: 290ms
6:	learn: 0.3194748	total: 5.72ms	remaining: 285ms
7:	learn: 0.3159140	total: 6.03ms	remaining: 262ms
8:	learn: 0.3120977	total: 6.44ms	remaining: 248ms
9:	learn: 0.3081473	total: 7.19ms	remaining: 249ms
10:	learn: 0.3046630	total: 7.91ms	remaining: 248ms
11:	learn: 0.3009902	total: 8.66ms	remaining: 248ms
12:	learn: 0.2976763	total: 9.42ms	remaining: 248ms
13:	learn: 0.2947095	total: 9.83ms	remaining: 240ms
14:	learn: 0.2916203	total: 10.6ms	remaining: 240ms
15:	learn: 0.2888269	total: 11.4ms	remaining: 241ms
16:	learn: 0.2859382	total: 12.1ms	remaining: 242ms
17:	learn: 0.2833146	total: 12.9ms	remaining: 242ms
18:	learn: 0.2806477	total: 13.7ms	remaining: 242ms
19:	learn: 0.2779312	to

189:	learn: 0.1847490	total: 142ms	remaining: 124ms
190:	learn: 0.1846047	total: 143ms	remaining: 124ms
191:	learn: 0.1842922	total: 144ms	remaining: 123ms
192:	learn: 0.1840644	total: 145ms	remaining: 122ms
193:	learn: 0.1838850	total: 146ms	remaining: 122ms
194:	learn: 0.1836019	total: 146ms	remaining: 121ms
195:	learn: 0.1834956	total: 147ms	remaining: 120ms
196:	learn: 0.1833372	total: 148ms	remaining: 119ms
197:	learn: 0.1829913	total: 148ms	remaining: 118ms
198:	learn: 0.1827906	total: 149ms	remaining: 118ms
199:	learn: 0.1826499	total: 150ms	remaining: 117ms
200:	learn: 0.1825661	total: 151ms	remaining: 116ms
201:	learn: 0.1823664	total: 151ms	remaining: 115ms
202:	learn: 0.1821656	total: 152ms	remaining: 115ms
203:	learn: 0.1819523	total: 153ms	remaining: 114ms
204:	learn: 0.1815900	total: 154ms	remaining: 113ms
205:	learn: 0.1812642	total: 154ms	remaining: 112ms
206:	learn: 0.1810272	total: 155ms	remaining: 112ms
207:	learn: 0.1807285	total: 156ms	remaining: 111ms
208:	learn: 

239:	learn: 0.1745709	total: 180ms	remaining: 87.1ms
240:	learn: 0.1743973	total: 181ms	remaining: 86.4ms
241:	learn: 0.1741607	total: 182ms	remaining: 85.7ms
242:	learn: 0.1739962	total: 183ms	remaining: 84.9ms
243:	learn: 0.1737642	total: 183ms	remaining: 84.2ms
244:	learn: 0.1734682	total: 184ms	remaining: 83.4ms
245:	learn: 0.1733459	total: 185ms	remaining: 82.7ms
246:	learn: 0.1730406	total: 186ms	remaining: 81.9ms
247:	learn: 0.1727626	total: 186ms	remaining: 81.1ms
248:	learn: 0.1724572	total: 187ms	remaining: 80.4ms
249:	learn: 0.1721962	total: 188ms	remaining: 79.6ms
250:	learn: 0.1720856	total: 188ms	remaining: 78.8ms
251:	learn: 0.1718984	total: 189ms	remaining: 78.1ms
252:	learn: 0.1717443	total: 190ms	remaining: 77.3ms
253:	learn: 0.1715601	total: 191ms	remaining: 76.6ms
254:	learn: 0.1713385	total: 191ms	remaining: 75.8ms
255:	learn: 0.1712226	total: 192ms	remaining: 75ms
256:	learn: 0.1709716	total: 193ms	remaining: 74.3ms
257:	learn: 0.1708374	total: 194ms	remaining: 73

3:	learn: 0.3293577	total: 21.3ms	remaining: 1.67s
4:	learn: 0.3274952	total: 26.5ms	remaining: 1.65s
5:	learn: 0.3257791	total: 27.1ms	remaining: 1.4s
6:	learn: 0.3237907	total: 31.6ms	remaining: 1.4s
7:	learn: 0.3218962	total: 33.2ms	remaining: 1.28s
8:	learn: 0.3201540	total: 33.8ms	remaining: 1.16s
9:	learn: 0.3185371	total: 38.4ms	remaining: 1.18s
10:	learn: 0.3168076	total: 43.1ms	remaining: 1.2s
11:	learn: 0.3149941	total: 47.6ms	remaining: 1.21s
12:	learn: 0.3132489	total: 52.4ms	remaining: 1.23s
13:	learn: 0.3115914	total: 57.1ms	remaining: 1.24s


14:	learn: 0.3100296	total: 61.9ms	remaining: 1.25s
15:	learn: 0.3084034	total: 66.6ms	remaining: 1.25s
16:	learn: 0.3067498	total: 71.2ms	remaining: 1.26s
17:	learn: 0.3050614	total: 75.6ms	remaining: 1.26s
18:	learn: 0.3034252	total: 80.3ms	remaining: 1.26s
19:	learn: 0.3019656	total: 85ms	remaining: 1.26s
20:	learn: 0.3005070	total: 89.6ms	remaining: 1.26s
21:	learn: 0.2989060	total: 93.9ms	remaining: 1.26s
22:	learn: 0.2975048	total: 98.5ms	remaining: 1.26s
23:	learn: 0.2959467	total: 103ms	remaining: 1.26s
24:	learn: 0.2945146	total: 108ms	remaining: 1.26s
25:	learn: 0.2932661	total: 108ms	remaining: 1.21s
26:	learn: 0.2917564	total: 113ms	remaining: 1.21s
27:	learn: 0.2903458	total: 117ms	remaining: 1.21s
28:	learn: 0.2890668	total: 122ms	remaining: 1.21s
29:	learn: 0.2876352	total: 127ms	remaining: 1.21s
30:	learn: 0.2862316	total: 131ms	remaining: 1.21s
31:	learn: 0.2849036	total: 136ms	remaining: 1.21s
32:	learn: 0.2835610	total: 141ms	remaining: 1.21s
33:	learn: 0.2821828	tot

...

222:	learn: 0.1817049	total: 1.01s	remaining: 425ms
223:	learn: 0.1815032	total: 1.01s	remaining: 421ms
224:	learn: 0.1812890	total: 1.02s	remaining: 416ms
225:	learn: 0.1810256	total: 1.02s	remaining: 412ms
226:	learn: 0.1808302	total: 1.03s	remaining: 407ms
227:	learn: 0.1805886	total: 1.03s	remaining: 403ms
228:	learn: 0.1803485	total: 1.04s	remaining: 398ms
229:	learn: 0.1800858	total: 1.04s	remaining: 394ms
230:	learn: 0.1798427	total: 1.05s	remaining: 390ms


231:	learn: 0.1796007	total: 1.05s	remaining: 385ms
232:	learn: 0.1794278	total: 1.05s	remaining: 381ms
233:	learn: 0.1791598	total: 1.06s	remaining: 376ms
234:	learn: 0.1789511	total: 1.06s	remaining: 372ms
235:	learn: 0.1787396	total: 1.07s	remaining: 367ms
236:	learn: 0.1785483	total: 1.07s	remaining: 362ms
237:	learn: 0.1783536	total: 1.08s	remaining: 358ms
238:	learn: 0.1781328	total: 1.08s	remaining: 354ms
239:	learn: 0.1779309	total: 1.09s	remaining: 349ms
240:	learn: 0.1777042	total: 1.09s	remaining: 345ms
241:	learn: 0.1774374	total: 1.1s	remaining: 340ms
242:	learn: 0.1772524	total: 1.1s	remaining: 336ms
243:	learn: 0.1770227	total: 1.11s	remaining: 331ms
244:	learn: 0.1768568	total: 1.11s	remaining: 327ms
245:	learn: 0.1766887	total: 1.12s	remaining: 322ms
246:	learn: 0.1764874	total: 1.12s	remaining: 318ms
247:	learn: 0.1762958	total: 1.13s	remaining: 313ms
248:	learn: 0.1761283	total: 1.13s	remaining: 309ms
249:	learn: 0.1759273	total: 1.13s	remaining: 304ms
250:	learn: 0.

265:	learn: 0.1728689	total: 1.21s	remaining: 232ms
266:	learn: 0.1726702	total: 1.21s	remaining: 227ms
267:	learn: 0.1724950	total: 1.22s	remaining: 223ms
268:	learn: 0.1723174	total: 1.22s	remaining: 218ms
269:	learn: 0.1721744	total: 1.23s	remaining: 214ms
270:	learn: 0.1720066	total: 1.23s	remaining: 209ms
271:	learn: 0.1718574	total: 1.24s	remaining: 204ms
272:	learn: 0.1716786	total: 1.24s	remaining: 200ms
273:	learn: 0.1714874	total: 1.25s	remaining: 195ms


274:	learn: 0.1713061	total: 1.25s	remaining: 191ms
275:	learn: 0.1711195	total: 1.25s	remaining: 186ms
276:	learn: 0.1709821	total: 1.26s	remaining: 182ms
277:	learn: 0.1708151	total: 1.26s	remaining: 177ms
278:	learn: 0.1706612	total: 1.27s	remaining: 173ms
279:	learn: 0.1705835	total: 1.27s	remaining: 168ms
280:	learn: 0.1704485	total: 1.28s	remaining: 164ms
281:	learn: 0.1702416	total: 1.28s	remaining: 159ms
282:	learn: 0.1700511	total: 1.29s	remaining: 155ms
283:	learn: 0.1698733	total: 1.29s	remaining: 150ms
284:	learn: 0.1697421	total: 1.29s	remaining: 145ms
285:	learn: 0.1695638	total: 1.3s	remaining: 141ms
286:	learn: 0.1693488	total: 1.3s	remaining: 136ms
287:	learn: 0.1692507	total: 1.31s	remaining: 132ms
288:	learn: 0.1690585	total: 1.31s	remaining: 127ms
289:	learn: 0.1689202	total: 1.32s	remaining: 123ms
290:	learn: 0.1687932	total: 1.32s	remaining: 118ms
291:	learn: 0.1686017	total: 1.33s	remaining: 114ms
292:	learn: 0.1684403	total: 1.33s	remaining: 109ms
293:	learn: 0.

308:	learn: 0.1658314	total: 1.4s	remaining: 36.3ms
309:	learn: 0.1657033	total: 1.41s	remaining: 31.8ms
310:	learn: 0.1655066	total: 1.41s	remaining: 27.3ms
311:	learn: 0.1653731	total: 1.42s	remaining: 22.7ms
312:	learn: 0.1652341	total: 1.42s	remaining: 18.2ms
313:	learn: 0.1650647	total: 1.43s	remaining: 13.6ms
314:	learn: 0.1649257	total: 1.43s	remaining: 9.08ms
315:	learn: 0.1647367	total: 1.44s	remaining: 4.54ms
316:	learn: 0.1645681	total: 1.44s	remaining: 0us


## Predict

In [23]:
sample = 'test'
preds_proba = {}
for task in TASKS:
    preds_proba[task] = models[task].predict(X[f'{sample}-{task}'])

    # Postprocessing
    np.clip(preds_proba[task], 0.0, 1.0, out=preds_proba[task])

    print(f'{sample}-{task} - done!')

test-agnostic - done!
test-aware - done!


# Submit
## Form dataframe

In [24]:
sample = 'test'
submission = {}
for task in TASKS:
    if USE_CATBOOST_CLASSIFIER:
        labels = preds_label[task]
    else:
        labels = pd.Series(preds_proba[task]).apply(lambda x: 'Hallucination' if x > 0.5 else 'Not Hallucination').to_list()
    
    submission[task] = pd.DataFrame({
        'id': X[f'{sample}-{task}'].index.to_list(),
        'label': labels,
        'p(Hallucination)': preds_proba[task],
        })
    display(submission[task])

,id,label,p(Hallucination)
0,1,Not Hallucination,0.285036
1,2,Not Hallucination,0.369860
2,3,Not Hallucination,0.184247
3,6,Hallucination,0.940665
4,7,Hallucination,0.566837
...,...,...,...
1495,2992,Not Hallucination,0.144588
1496,2993,Hallucination,0.897099
1497,2994,Not Hallucination,0.335924
1498,2996,Not Hallucination,0.166182


,id,label,p(Hallucination)
0,0,Not Hallucination,0.153799
1,4,Not Hallucination,0.306519
2,5,Not Hallucination,0.448077
3,9,Hallucination,0.775503
4,11,Not Hallucination,0.123365
...,...,...,...
1495,2988,Not Hallucination,0.474919
1496,2991,Not Hallucination,0.372153
1497,2995,Not Hallucination,0.149408
1498,2997,Not Hallucination,0.198539


## Save to json

In [25]:
sample = 'test'
for task in TASKS:
    json_file_path = f'{SUBMISSION_PATH}{sample}.model-{task}.json'

    # Check if file exists
    assert not os.path.exists(json_file_path), 'File already exists.'

    # Save to .json file
    submission[task].to_json(path_or_buf=json_file_path, orient='records')

    print(json_file_path)

./output/catboost/test6/test.model-agnostic.json
./output/catboost/test6/test.model-aware.json


## Check file format

In [26]:
sample = 'test'
if sample == 'val':
    !python ./check_output.py {SUBMISSION_PATH} --is_val
elif sample == 'test':
    !python ./check_output.py {SUBMISSION_PATH}

all clear!


## Evaluate

In [27]:
sample = 'test'
if sample == 'val':
    !python ./score.py {SUBMISSION_PATH} {REFERENCE_PATH} {RESULT_PATH} --is_val
elif sample == 'test':
    !python ./score.py {SUBMISSION_PATH} {REFERENCE_PATH} {RESULT_PATH}

with open(RESULT_PATH) as fp:
    print(fp.read())

agnostic_acc:0.8253333333333334
agnostic_rho:0.7571334944036751
aware_acc:0.7993333333333333
aware_rho:0.7217873788633554



# Feature importance

In [28]:
for task in TASKS:
    print(task)
    display(pd.DataFrame({
        'feature_importance': models[task].get_feature_importance(), 
        'feature_names': X[f'val-{task}'].columns,
    }).sort_values(by=['feature_importance'], ascending=False))

agnostic


,feature_importance,feature_names
5,15.161205,p(Contr)_google_t5_xxl_true_nli_mixture
6,9.723069,p(Contr)_sentence-transformers-nli-roberta-large
16,8.870520,p(Entl)_microsoft-deberta-v2-xxlarge-mnli
3,7.117579,p(Contr)_FacebookAI-roberta-large-mnli
2,6.703525,src_len
8,6.317207,p(Entl)_microsoft-deberta-base-mnli
14,5.264956,p(Entl)_microsoft-deberta-v2-xlarge-mnli
12,5.149985,p(Entl)_microsoft-deberta-xlarge-mnli
1,4.458982,tgt_len
15,4.426080,p(Contr)_microsoft-deberta-v2-xxlarge-mnli


aware


,feature_importance,feature_names
6,18.408850,p(Contr)_sentence-transformers-nli-roberta-large
4,8.687278,p(Contr)_deberta-selfchecknli
1,6.915558,tgt_len
0,6.913941,hyp_len
2,5.900871,src_len
5,5.689571,p(Contr)_google_t5_xxl_true_nli_mixture
10,5.351988,p(Entl)_microsoft-deberta-large-mnli
16,5.144338,p(Entl)_microsoft-deberta-v2-xxlarge-mnli
3,4.894154,p(Contr)_FacebookAI-roberta-large-mnli
12,4.851402,p(Entl)_microsoft-deberta-xlarge-mnli
